# Setting Environment Up

First, we need to install the package and all the requirements. Note that we do not need to install pytorch and torchvision as google colab comes with its own GPU-optimized packages.

In [ ]:
!git clone https://github.com/DiegoZertuche/BertLearningDynamics.git

Cloning into 'BertLearningDynamics'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (362/362), done.
remote: Compressing objects: 100% (266/266), done.
remote: Total 362 (delta 128), reused 317 (delta 88), pack-reused 0
Receiving objects: 100% (362/362), 1.13 MiB | 6.22 MiB/s, done.
Resolving deltas: 100% (128/128), done.


In [ ]:
%cd BertLearningDynamics/

/content/BertLearningDynamics


In [ ]:
!pip install -r requirements-no-torch.txt

     |████████████████████████████████| 255 kB 9.3 MB/s 
     |████████████████████████████████| 6.3 MB 28.9 MB/s 
     |████████████████████████████████| 147 kB 53.4 MB/s 
     |████████████████████████████████| 1.5 MB 24.0 MB/s 
     |████████████████████████████████| 162 kB 46.2 MB/s 
     |████████████████████████████████| 20.2 MB 1.3 MB/s 
     |████████████████████████████████| 10.0 MB 19.3 MB/s 
     |████████████████████████████████| 50 kB 6.7 MB/s 
     |████████████████████████████████| 883 kB 47.4 MB/s 
     |████████████████████████████████| 7.1 MB 35.1 MB/s 
     |████████████████████████████████| 1.1 MB 39.8 MB/s 
     |████████████████████████████████| 3.2 MB 31.2 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 2.1 MB 44.0 MB/s 
     |████████████████████████████████| 738 kB 40.4 MB/s 
     |████████████████████████████████| 114 kB 44.7 MB/s 
     |████████████████████████████████| 243 kB 46.3 MB/s 
     |████████████

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Automatization

**RESTART RUNTIME AT THIS POINT AND CHANGE TOKENIZER**

In [ ]:
%cd /content/BertLearningDynamics

/content/BertLearningDynamics


In [ ]:
from main_model import MainModel
import torch
from torch.utils.data import DataLoader
from preprocess import MyDataset
from torch.nn.utils.rnn import pad_sequence
from torch.nn.functional import softmax
from tqdm import tqdm
from allennlp.training.metrics import F1MultiLabelMeasure
import numpy as np
from google.colab import files
import pickle
#import torch_xla.core.xla_model as xm

In [ ]:
params = {
    'input_module': 'diegozs97/finetuned-chemprot-seed-0-0k',
    'transfer_mode': 'frozen',
    'output_mode': 'mix',
    'max_layer': -1
}

task_params = {
    'span_pooling': 'attn',
    'n_spans': 2,
    'n_classes': 2,
    'single_sided': False,
    'cls_type': 'mlp',
    'dropout': 0.2
}

#model = MainModel(params, task_params, device='tpu')
model = MainModel(params, task_params, is_cuda=True)

In [ ]:
def update_metrics(out, f1_scorer):
    preds = out["preds"]
    labels = out["labels"]
    f1_scorer(preds, labels)


def tokenize_data(texts):
    return model.encoder.tokenizer.batch_encode_plus(texts, return_tensors='pt', padding=True)


def pad_collate(batch):
    (xx, spans1, spans2, labels) = zip(*batch)
    xx_pad = tokenize_data(xx)
    spans1_padded = pad_sequence(spans1, batch_first=True, padding_value=-1)
    spans2_padded = pad_sequence(spans2, batch_first=True, padding_value=-1)
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=-1)
    return xx_pad, spans1_padded, spans2_padded, labels_padded

In [ ]:
train_task_filepath = '/content/drive/MyDrive/Google_2021_Capstone/data/ontonotes/coref/train.json'
labels_filepath = '/content/drive/MyDrive/Google_2021_Capstone/data/ontonotes/coref/labels.txt'
train_ds = MyDataset(train_task_filepath, train_task_filepath + '.retokenized.bert-base-uncased', labels_filepath, n_spans=2)
dev_task_filepath = '/content/drive/MyDrive/Google_2021_Capstone/data/ontonotes/coref/development.json'
dev_ds = MyDataset(dev_task_filepath, dev_task_filepath + '.retokenized.bert-base-uncased', labels_filepath, n_spans=2)

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True, drop_last=True, collate_fn=pad_collate)
dev_dl = DataLoader(dev_ds, batch_size=32, shuffle=True, drop_last=True, collate_fn=pad_collate)

In [ ]:
multibert_string = "diegozs97/finetuned-chemprot-seed-"
seeds = [str(x) for x in range(25)]
checkpoints = ['0k', '20k', '60k', '100k', '200k', '400k', 
                   '700k', '1000k', '1500k', '1800k', '2000k']

In [ ]:
def log_and_save(losses, metrics, filename):
  with open(filename+'-losses.txt', 'w') as f:
    for loss in losses:
      f.write('{}'.format(loss))
      f.write('\n')
    f.close()

  with open(filename+'-metrics.txt', 'w') as f:
    for metric in metrics:
      f.write('{} {} {}'.format(metric['precision'], metric['recall'], metric['fscore']))
      f.write('\n')
    f.close()

In [ ]:
def run_batch(model_strings, filenames, epochs, dev):
  for model_string, filename in zip(model_strings, filenames):
    print('='*30)
    params = {
    'input_module': model_string,
    'transfer_mode': 'frozen',
    'output_mode': 'mix',
    'max_layer': -1
    }

    task_params = {
        'span_pooling': 'attn',
        'n_spans': 2,
        'n_classes': 2,
        'single_sided': False,
        'cls_type': 'mlp',
        'dropout': 0.2
    }
    model = MainModel(params, task_params, is_cuda=dev)

    optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
    f1_scorer = F1MultiLabelMeasure(average="micro")

    train_losses = []
    val_losses = []
    train_metrics = []
    val_metrics = []
    n_epochs = epochs
    print('Training model {}'.format(model_string))
    for epoch in range(n_epochs):
        running_loss = 0
        f1_scorer.reset()
        for batch in tqdm(train_dl, leave=False):
            outs = model(batch, True)
            outs['loss'].backward()
            optimizer.step()
            optimizer.zero_grad()
            running_loss += outs['loss']
            
            update_metrics(outs, f1_scorer)

        train_losses.append(running_loss)
        train_metrics.append(f1_scorer.get_metric())
        print("=" * 20)
        print(f"Epoch {epoch + 1}/{n_epochs} Train Loss: {running_loss}")
        print(f"Epoch {epoch + 1}/{n_epochs} Train Metrics: {f1_scorer.get_metric()}")

        running_loss = 0
        f1_scorer.reset()
        with torch.no_grad():
            for batch in tqdm(dev_dl, leave=False):
                outs = model(batch, True)
                running_loss += outs['loss'].item()
                update_metrics(outs, f1_scorer)

        val_losses.append(running_loss)
        val_metrics.append(f1_scorer.get_metric())
        print(f"Epoch {epoch + 1}/{n_epochs} Val Loss: {running_loss}")
        print(f"Epoch {epoch + 1}/{n_epochs} Val Metrics: {f1_scorer.get_metric()}")
    torch.save(model.state_dict(), '/content/drive/MyDrive/Google_2021_Capstone/results/coref/seed-0/{}-dict.pth'.format(filename))
    #log_and_save(train_losses, train_metrics, '/content/drive/MyDrive/results/ontonotes/coref/seed-0/{}-train'.format(filename))
    log_and_save(val_losses, val_metrics, '/content/drive/MyDrive/Google_2021_Capstone/results/coref/seed-0/{}-val'.format(filename))
        

In [ ]:
model_list = [multibert_string+seeds[0]+'-'+check for check in checkpoints]
filenames = checkpoints

In [ ]:
run_batch(model_list, filenames, 20, True)

  0%|          | 0/1305 [00:00<?, ?it/s]

Training model diegozs97/finetuned-chemprot-seed-0-200k


  0%|          | 0/157 [00:00<?, ?it/s]

Epoch 1/20 Train Loss: 479.14910888671875
Epoch 1/20 Train Metrics: {'precision': 0.8489506840705872, 'recall': 0.8494491577148438, 'fscore': 0.8491998910903931}


  0%|          | 0/1305 [00:00<?, ?it/s]

...

  0%|          | 0/157 [00:00<?, ?it/s]

Epoch 18/20 Train Loss: 123.4222183227539
Epoch 18/20 Train Metrics: {'precision': 0.9638091921806335, 'recall': 0.9640411138534546, 'fscore': 0.9639251828193665}


  0%|          | 0/1305 [00:00<?, ?it/s]

Epoch 18/20 Val Loss: 27.349183604121208
Epoch 18/20 Val Metrics: {'precision': 0.9347991943359375, 'recall': 0.9341939687728882, 'fscore': 0.9344964623451233}


 13%|█▎        | 173/1305 [00:57<05:45,  3.28it/s]